In [3]:
import json
from pathlib import Path

from histalign.backend.models import (
    AlignmentSettings,
    HistologySettings,
    ProjectSettings,
    VolumeSettings,
)
from histalign.backend.workspace import HistologySlice, Workspace

In [4]:
old_directory = Path(
    "/home/ediun/git/histalign/projects/project_310724_full.bak/7d9af388b0"
)
new_directory = Path(
    "/home/ediun/git/histalign/projects/project_310724_full/7d9af388b0"
)

## project.json

In [5]:
with open(old_directory.parent / "project.json") as handle:
    contents = json.load(handle)

project_settings = ProjectSettings(
    project_path=contents["project_directory_path"],
    orientation="sagittal",
    resolution=contents["atlas_resolution"],
)

workspace = Workspace(project_settings)
workspace.working_directory = "/home/ediun/git/histalign/projects/project_310724_full"
workspace.last_parsed_directory = contents["last_parsed_directory"]
workspace.current_aligner_image_hash = contents["current_aligner_image_hash"]
workspace.current_aligner_image_index = contents["current_aligner_image_index"]

workspace.save()

## alignment.json

In [7]:
for file in Path(old_directory).iterdir():
    if file.suffix != ".json" or file.stem == "metadata":
        continue

    with open(file) as handle:
        contents = json.load(handle)

    alignment_settings = AlignmentSettings(
        volume_path=contents["volume_file_path"],
        volume_scaling=contents["volume_scaling_factor"],
        volume_settings=VolumeSettings(
            orientation="sagittal",
            resolution=contents["resolution"],
            shape=[
                528,
                320,
                456,
            ],
            pitch=contents["leaning_angle"],
            yaw=0,
            offset=contents["offset"],
        ),
        histology_path=contents["histology_file_path"],
        histology_scaling=contents["histology_scaling_factor"],
        histology_downsampling=contents["downsampling_factor"],
        histology_settings=HistologySettings(
            rotation=contents["rotation_angle"],
            translation_x=contents["x_translation"],
            translation_y=contents["y_translation"],
            scale_x=contents["x_scale"],
            scale_y=contents["y_scale"],
            shear_x=contents["x_shear"],
            shear_y=contents["y_shear"],
        ),
    )

    histology_slice = HistologySlice(contents["histology_file_path"])

    workspace.alignment_settings = alignment_settings
    workspace.current_aligner_image_hash = histology_slice.hash
    workspace.save_alignment()

## metadata.json

In [28]:
with open(old_directory / "metadata.json") as handle:
    metadata = json.load(handle)